In [2]:
%pip install mlflow>=2.11.0
%pip install transformers peft accelerate bitsandbytes datasets -q -U

In [14]:
from datasets import Dataset

with open("/content/David_Copperfield.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Remove empty lines and strip
lines = [line.strip() for line in lines if line.strip()]
window_size = 3

beginning = []
continuation = []

for i in range(len(lines) - window_size):
    # Take first part as prompt, rest as completion
    beginning.append(" ".join(lines[i:i+2]))
    continuation.append(" ".join(lines[i+2:i+window_size+1]))

# Create a Dataset from a list of dicts
dataset = Dataset.from_dict({"beginning": beginning, "continuation": continuation})

In [17]:
import pandas as pd
from datasets import load_dataset
from IPython.display import HTML, display

def display_table(dataset_or_sample):
  # A helper fuction to display a Transformer dataset or single sample contains multi-line string nicely
  pd.set_option("display.max_colwidth", None)
  pd.set_option("display.width", None)
  pd.set_option("display.max_rows", None)

  if isinstance(dataset_or_sample, dict):
      df = pd.DataFrame(dataset_or_sample, index=[0])
  else:
      df = pd.DataFrame(dataset_or_sample)

  html = df.to_html()
  styled_html = f"""<style> .dataframe th, .dataframe tbody td {{ text-align: left; padding-right: 30px; }} </style> {html}"""
  display(HTML(styled_html))


display_table(dataset.select(range(10)))

,beginning,continuation
0,"By Charles Dickens. This ebook is the product of many hours of hard work by volunteers for Standard Ebooks, and builds on the hard work of other literature lovers made possible by the public domain.","This particular ebook is based on a transcription from Project Gutenberg and on digital scans from the Internet Archive. The source text and artwork in this ebook are believed to be in the United States public domain; that is, they are believed to be free of copyright restrictions in the United States. They may still be copyrighted in other countries, so users located outside of the United States must check their local laws before using this ebook. The creators of, and contributors to, this ebook dedicate their contributions to the worldwide public domain via the terms in the CC0 1.0 Universal Public Domain Dedication. For full license information, see the Uncopyright at the end of this ebook."
1,"This ebook is the product of many hours of hard work by volunteers for Standard Ebooks, and builds on the hard work of other literature lovers made possible by the public domain. This particular ebook is based on a transcription from Project Gutenberg and on digital scans from the Internet Archive.","The source text and artwork in this ebook are believed to be in the United States public domain; that is, they are believed to be free of copyright restrictions in the United States. They may still be copyrighted in other countries, so users located outside of the United States must check their local laws before using this ebook. The creators of, and contributors to, this ebook dedicate their contributions to the worldwide public domain via the terms in the CC0 1.0 Universal Public Domain Dedication. For full license information, see the Uncopyright at the end of this ebook. Standard Ebooks is a volunteer-driven project that produces ebook editions of public domain literature using modern typography, technology, and editorial standards, and distributes them free of cost. You can download this and other ebooks carefully produced for true book lovers at standardebooks.org."
2,"This particular ebook is based on a transcription from Project Gutenberg and on digital scans from the Internet Archive. The source text and artwork in this ebook are believed to be in the United States public domain; that is, they are believed to be free of copyright restrictions in the United States. They may still be copyrighted in other countries, so users located outside of the United States must check their local laws before using this ebook. The creators of, and contributors to, this ebook dedicate their contributions to the worldwide public domain via the terms in the CC0 1.0 Universal Public Domain Dedication. For full license information, see the Uncopyright at the end of this ebook.","Standard Ebooks is a volunteer-driven project that produces ebook editions of public domain literature using modern typography, technology, and editorial standards, and distributes them free of cost. You can download this and other ebooks carefully produced for true book lovers at standardebooks.org. I do not find it easy to get sufficiently far away from this book, in the first sensations of having finished it, to refer to it with the composure which this formal heading would seem to require. My interest in it, is so recent and strong; and my mind is so divided between pleasure and regret⁠—pleasure in the achievement of a long design, regret in the separation from many companions⁠—that I am in danger of wearying the reader whom I love, with personal confidences, and private emotions."
3,"The source text and artwork in this ebook are believed to be in the United States public domain; that is, they are believed to be free of copyright restrictions in the United States. They may still be copyrighted in other countries, so users located outside of the United States must check their local laws before using this ebook. The creators of, and contributors to, 

In [18]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Training dataset contains {len(train_dataset)}")
print(f"Test dataset contains {len(test_dataset)}")

Training dataset contains 5844
Test dataset contains 1461


In [19]:
DICKENS_STYLE_TEMPLATE = """You are a Victorian novelist trained in Charles Dickens' writing style. Continue the given passage in the distinctive voice and style of David Copperfield.

### Beginning:
{passage_start}

### Continuation:
{dickens_continuation}"""

def apply_dickens_template(row):
    prompt = DICKENS_STYLE_TEMPLATE.format(
        passage_start=row['beginning'],
        dickens_continuation=row['continuation']
    )
    return {"prompt": prompt}


train_dataset = train_dataset.map(apply_dickens_template)
display_table(train_dataset.select(range(1)))

Map:   0%|          | 0/5844 [00:00<?, ? examples/s]

,beginning,continuation,prompt
0,"“Far from it, ma’am,” I returned. “Far from it,” assented Miss Murdstone. “I do not wish to revive the memory of past differences, or of past outrages. I have received outrages from a person⁠—a female I am sorry to say, for the credit of my sex⁠—who is not to be mentioned without scorn and disgust; and therefore I would rather not mention her.”","I felt very fiery on my aunt’s account; but I said it would certainly be better, if Miss Murdstone pleased, not to mention her. I could not hear her disrespectfully mentioned, I added, without expressing my opinion in a decided tone. Miss Murdstone shut her eyes, and disdainfully inclined her head; then, slowly opening her eyes, resumed:","You are a Victorian novelist trained in Charles Dickens' writing style. Continue the given passage in the distinctive voice and style of David Copperfield.\n\n### Beginning:\n“Far from it, ma’am,” I returned. “Far from it,” assented Miss Murdstone. “I do not wish to revive the memory of past differences, or of past outrages. I have received outrages from a person⁠—a female I am sorry to say, for the credit of my sex⁠—who is not to be mentioned without scorn and disgust; and therefore I would rather not mention her.”\n\n### Continuation:\nI felt very fiery on my aunt’s account; but I said it would certainly be better, if Miss Murdstone pleased, not to mention her. I could not hear her disrespectfully mentioned, I added, without expressing my opinion in a decided tone. Miss Murdstone shut her eyes, and disdainfully inclined her head; then, slowly opening her eyes, resumed:"


In [7]:
from huggingface_hub import login
login()

In [20]:
from transformers import AutoTokenizer

base_model_id = "meta-llama/Llama-3.2-3B-Instruct"

# You can use a different max length if your custom dataset has shorter/longer input sequences.
MAX_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(
  base_model_id,
  model_max_length=MAX_LENGTH,
  padding_side="left",
  add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token


def tokenize_and_pad_to_fixed_length(sample):
  result = tokenizer(
      sample["beginning"],
      truncation=True,
      max_length=MAX_LENGTH,
      padding="max_length",
  )
  result["labels"] = result["input_ids"].copy()
  return result


tokenized_train_dataset = train_dataset.map(tokenize_and_pad_to_fixed_length)

assert all(len(x["input_ids"]) == MAX_LENGTH for x in tokenized_train_dataset)

display_table(tokenized_train_dataset.select(range(1)))

Map:   0%|          | 0/5844 [00:00<?, ? examples/s]

,beginning,continuation,prompt,input_ids,attention_mask,labels
0,"“Far from it, ma’am,” I returned. “Far from it,” assented Miss Murdstone. “I do not wish to revive the memory of past differences, or of past outrages. I have received outrages from a person⁠—a female I am sorry to say, for the credit of my sex⁠—who is not to be mentioned without scorn and disgust; and therefore I would rather not mention her.”","I felt very fiery on my aunt’s account; but I said it would certainly be better, if Miss Murdstone pleased, not to mention her. I could not hear her disrespectfully mentioned, I added, without expressing my opinion in a decided tone. Miss Murdstone shut her eyes, and disdainfully inclined her head; then, slowly opening her eyes, resumed:","You are a Victorian novelist trained in Charles Dickens' writing style. Continue the given passage in the distinctive voice and style of David Copperfield.\n\n### Beginning:\n“Far from it, ma’am,” I returned. “Far from it,” assented Miss Murdstone. “I do not wish to revive the memory of past differences, or of past outrages. I have received outrages from a person⁠—a female I am sorry to say, for the credit of my sex⁠—who is not to be mentioned without scorn and disgust; and therefore I would rather not mention her.”\n\n### Continuation:\nI felt very fiery on my aunt’s account; but I said it would certainly be better, if Miss Murdstone pleased, not to mention her. I could not hear her disrespectfully mentioned, I added, without expressing my opinion in a decided tone. Miss Murdstone shut her eyes, and disdainfully inclined her head; then, slowly opening her eyes, resumed:","[128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128000, 2118, 33987, 505, 433, 11, 7643, 77138, 2476, 358, 6052, 13, 1054, 33987, 505, 433, 2476, 1089, 16243, 9083, 15356, 67, 11046, 13, 1054, 40, 656, 539, 6562, 311, 59816, 279, 5044, 315, 3347, 12062, 11, 477, 315, 3347, 18333, 1154, 13, 358, 617, 4036, 18333, 1154, 505, 264, 1732, 53233, 254, 2345, 64, 8954, 358, 1097, 14931, 311, 2019, 11, 369, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128000, 2118, 33987, 505, 433, 11, 7643, 77138, 2476, 358, 6052, 13, 1054, 33987, 505, 433, 2476, 1089, 16243, 9083, 15356, 67, 11046, 13, 1054, 40, 656, 539, 6562, 311, 59816, 279, 5044, 315, 3347, 12062, 11, 477, 315, 3347, 18333, 1154, 13, 358, 617, 4036, 18333, 1154, 505, 264, 1732, 53233, 254, 2345, 64, 8954, 358, 1097, 14931, 311, 2019, 11, 369, ...]"


In [21]:
pip install -U bitsandbytes

In [22]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
  # Load the model with 4-bit quantization
  load_in_4bit=True,
  # Use double quantization
  bnb_4bit_use_double_quant=True,
  # Use 4-bit Normal Float for storing the base model weights in GPU memory
  bnb_4bit_quant_type="nf4",
  # De-quantize the weights to 16-bit (Brain) float before the forward/backward pass
  bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=quantization_config)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [24]:
import transformers

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipeline = transformers.pipeline(model=model, tokenizer=tokenizer, task="text-generation")

sample = test_dataset[1]
prompt = DICKENS_STYLE_TEMPLATE.format(
          passage_start=sample['beginning'],
        dickens_continuation=""
)  # Leave the answer part blank

with torch.no_grad():
  response = pipeline(prompt, max_new_tokens=128, repetition_penalty=1.15, return_full_text=False)

display_table({"dickens_continuation": test_dataset[1]['continuation'], "generated_query": response[0]["generated_text"]})

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


,dickens_continuation,generated_query
0,"Though he anxiously endeavoured to dissuade me, I saw that he was of my mind; and this, if I had required to be confirmed in my intention, would have had the effect. He went round to the coach office, at my request, and took the box-seat for me on the mail. In the evening I started, by that conveyance, down the road I had traversed under so many vicissitudes. “Don’t you think that,” I asked the coachman, in the first stage out of London, “a very remarkable sky? I don’t remember to have seen one like it.”","“Mas’r Davy” was gone, and I had taken the reins of my own destiny once more. As I descended into the bustling streets, the fog rolled up about me like a damp shroud, obscuring all familiar landmarks and making every step a gamble with uncertainty. But I moved forward undaunted, driven by the indomitable spirit that had characterized my life thus far – that great engine which had hauled me out of the mire of childhood poverty and sent me forth into an uncertain but boundless world.\n\nAnd then as if Providence itself were nudgeing me toward some hidden purpose, I chanced upon"


In [25]:
with torch.no_grad():
  response = pipeline("my deepwst sorrows are", max_new_tokens=128, repetition_penalty=1.15, return_full_text=False)

display_table({"prompt": "my deepwst sorrows are", "generated_query": response[0]["generated_text"]})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


,prompt,generated_query
0,my deepwst sorrows are,"soothed by the sight of you, and my heart is lifted when I see your lovely face.\nYou are a shining light in a world filled with darkness, a beacon of hope in times of despair. Your presence calms me, and brings peace to my soul.\n\nI am forever changed by our time together, and I will carry the memories of our moments shared with me always. You have shown me that even in the most trying of times, there is beauty to be found, and love to be experienced.\n\nThank you for being my guiding star, my safe haven, and my forever home. I promise to cherish and honor you"


In [26]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Enabling gradient checkpointing, to make the training further efficient
model.gradient_checkpointing_enable()
# Set up the model for quantization-aware training e.g. casting layers, parameter freezing, etc.
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
  task_type="CAUSAL_LM",
  # This is the rank of the decomposed matrices A and B to be learned during fine-tuning. A smaller number will save more GPU memory but might result in worse performance.
  r=32,
  # This is the coefficient for the learned ΔW factor, so the larger number will typically result in a larger behavior change after fine-tuning.
  lora_alpha=64,
  # Drop out ratio for the layers in LoRA adaptors A and B.
  lora_dropout=0.1,
  # We fine-tune all linear layers in the model. It might sound a bit large, but the trainable adapter size is still only **1.16%** of the whole model.
  target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
      "lm_head",
  ],
  # Bias parameters to train. 'none' is recommended to keep the original model performing equally when turning off the adapter.
  bias="none",
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 52,830,208 || all params: 3,265,580,032 || trainable%: 1.6178


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [27]:
from datetime import datetime

import transformers
from transformers import TrainingArguments

import mlflow

# Comment-out this line if you are running the tutorial on Databricks
mlflow.set_experiment("MLflow PEFT Tutorial")

training_args = TrainingArguments(
  # Set this to mlflow for logging your training
  report_to="mlflow",
  # Name the MLflow run
  run_name=f"Llama-3.2-3B-Instruct-Dickens-QLoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
  # Replace with your output destination
  output_dir=".",
  # For the following arguments, refer to https://huggingface.co/docs/transformers/main_classes/trainer
  per_device_train_batch_size=2,
  gradient_accumulation_steps=4,
  gradient_checkpointing=True,
  optim="paged_adamw_8bit",
  bf16=True,
  learning_rate=2e-5,
  lr_scheduler_type="constant",
  max_steps=50,
  save_steps=10,
  logging_steps=10,
  warmup_steps=5,
  # https://discuss.huggingface.co/t/training-llama-with-lora-on-multiple-gpus-may-exist-bug/47005/3
  ddp_find_unused_parameters=False,
)

trainer = transformers.Trainer(
  model=peft_model,
  train_dataset=tokenized_train_dataset,
  data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
  args=training_args,
)

# use_cache=True is incompatible with gradient checkpointing.
peft_model.config.use_cache = False

2025/06/08 09:27:26 INFO mlflow.tracking.fluent: Experiment with name 'MLflow PEFT Tutorial' does not exist. Creating a new experiment.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
tokenizer.pad_token = tokenizer.eos_token

In [29]:
trainer.train()

Step,Training Loss
10,3.449900
20,3.267700
30,3.181900
40,3.057900
50,3.030000


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=50, training_loss=3.197480354309082, metrics={'train_runtime': 1053.6614, 'train_samples_per_second': 0.38, 'train_steps_per_second': 0.047, 'total_flos': 882148638720000.0, 'train_loss': 3.197480354309082, 'epoch': 0.06844626967830253})

In [33]:
# Basically the same format as we applied to the dataset. However, the template only accepts {prompt} variable so both table and question need to be fed in there.
prompt_template = """You are a Victorian novelist trained in Charles Dickens' writing style. Continue the given passage in the distinctive voice and style of David Copperfield.

### Beginning:
{prompt}

### Continuation:
"""

In [31]:
from mlflow.models import infer_signature

sample = train_dataset[1]

# MLflow infers schema from the provided sample input/output/params
signature = infer_signature(
  model_input=sample["beginning"],
  model_output=sample["continuation"],
  # Parameters are saved with default values if specified
  params={"max_new_tokens": 128, "repetition_penalty": 1.15, "return_full_text": False},
)
signature

inputs: 
  [string (required)]
outputs: 
  [string (required)]
params: 
  ['max_new_tokens': long (default: 128), 'repetition_penalty': double (default: 1.15), 'return_full_text': boolean (default: False)]

In [34]:
import mlflow

# Get the ID of the MLflow Run that was automatically created above
last_run_id = mlflow.last_active_run().info.run_id
print(last_run_id)

# Save a tokenizer without padding because it is only needed for training
tokenizer_no_pad = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True)

# If you interrupt the training, uncomment the following line to stop the MLflow run
# mlflow.end_run()

with mlflow.start_run(run_id=last_run_id):
  mlflow.log_params(peft_config.to_dict())
  mlflow.transformers.log_model(
      transformers_model={"model": trainer.model, "tokenizer": tokenizer_no_pad},
      prompt_template=prompt_template,
      signature=signature,
      artifact_path="model",  # This is a relative path to save model files within MLflow run
  )

0a7408f9615b4d0c87926c821820142e


Device set to use cuda:0
2025/06/08 09:46:18 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor and config will be saved, but the base model weights will not and reference to the HuggingFace Hub repository will be logged instead.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
2025/06/08 09:48:09 INFO mlflow.transformers: Skipping saving pretrained model weights to disk as the save_pretrained argumentis set to False. The reference to the HuggingFace Hub repository meta-llama/Llama-3.2-3B-Instruct will be logged instead.


README.md:   0%|          | 0.00/41.7k [00:00<?, ?B/s]

LICENSE.txt:   0%|          | 0.00/7.71k [00:00<?, ?B/s]

2025/06/08 09:48:10 INFO mlflow.transformers: text-generation pipelines saved with prompt templates have the `return_full_text` pipeline kwarg set to False by default. To override this behavior, provide a `model_config` dict with `return_full_text` set to `True` when saving the model.
2025/06/08 09:48:10 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/08 09:48:10 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version la

In [ ]:
import mlflow

# You can find the ID of run in the Run detail page on MLflow UI
mlflow_model = mlflow.pyfunc.load_model("runs:/0a7408f9615b4d0c87926c821820142e/model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
2025/06/08 10:00:52 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# We only input table and question, since system prompt is adeed in the prompt template.
test_prompt = """
my deepest sorrows are
"""

# Inference parameters like max_tokens_length are set to default values specified in the Model Signature
generated_query = mlflow_model.predict(test_prompt)[0]
display_table({"prompt": test_prompt, "generated_query": generated_query})